#Link to google colab notebook
https://colab.research.google.com/drive/1PezURxs-ZiyHiKzG0GxprplIggjYBPHC?usp=sharing

Importing required Libraries

In [62]:
import pandas as pd
#import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [63]:
import string
import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ram_o\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ram_o\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ram_o\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ram_o\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ram_o\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Importing Dataset

In [64]:
df = pd.read_csv('gne-release-v1.0.tsv', sep='\t')
df

,id,headline,url,bias_horizontal,bias_vertical,country,source,dominant_emotion,intensity,cause,experiencer,target,cue,other_emotions,reader_emotions
0,7d3fe468,Cops in One Village Have Been Convicted of 70 ...,http://tracking.feedpress.it/link/9499/12677255,-5.0,46.0,US,ProPublica,anger,medium,"[[""cops in one village have been convicted of ...","[[""cops""]]","[[""cops in one village have been convicted of ...","[[""convicted""]]",annoyance,annoyance
1,86693d59,DIY penis enlargements are a 'nationwide probl...,https://www.dailymail.co.uk/news/article-69149...,13.0,19.0,UK,dailymail,negative_surprise,medium,"[[""diy penis enlargements are a 'nationwide pr...","[[""papua new guinea""]]","[[""diy penis enlargements are a 'nationwide pr...","[[""problem""]]",negative_surprise,no_i_did_not_feel_any_emotion
2,0fb40e90,Dam breaking: New Epstein accuser comes forward,https://hotair.com/archives/ed-morrissey/2019/...,NaN,NaN,US,HotAir,anger,medium,"[[""new epstein accuser comes forward""]]",[[]],"[[""new epstein accuser comes forward""]]","[[""accuser""]]",no_other_additional_emotions_are_expressed,no_i_did_not_feel_any_emotion
3,fa7750d6,David Beckham gets six-month driving ban for u...,https://www.theguardian.com/football/2019/may/...,-6.0,48.0,UK,TheGuardian,negative_surprise,medium,"[[""gets six-month driving ban for using phone ...","[[""david beckham""]]","[[""driving ban""]]","[[""ban""]]",no_other_additional_emotions_are_expressed,no_i_did_not_feel_any_emotion
4,695ea7a2,Dead sea turtle found with spear through head ...,https://www.foxnews.com/science/sea-turtle-spe...,27.0,20.0,US,FoxNews,sadness,medium,"[[""dead sea turtle found with spear through he...","[[""florida national park""]]","[[""dead sea turtle found with spear through he...","[[""dead""]]",no_other_additional_emotions_are_expressed,sadness
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3cb09e5b,‘NOW do abortion. We’ll wait.’ Ilhan Omar doub...,https://twitchy.com/samj-3930/2019/06/16/now-d...,29.0,14.0,US,Twitchy,negative_surprise,medium,"[[""on her gun violence claim and only makes he...","[[""ilhan omar""]]","[[""ilhan omar""]]","[[""violence"",""doubles down""]]",negative_surprise,no_i_did_not_feel_any_emotion
4996,86f400a6,‘Triad’ Thugs Use Clubs to Punish Hong Kong’s ...,https://www.thedailybeast.com/in-hong-kong-tri...,-21.0,41.0,US,dailybeast,negative_surprise,medium,"[[""‘triad’ thugs use clubs to punish hong kong...",[[]],"[[""‘triad’ thugs use clubs to punish hong kong...","[[""punish""]]",no_other_additional_emotions_are_expressed,no_i_did_not_feel_any_emotion
4997,99ca38ca,'For real': High school football season begins...,https://www.theadvocate.com/baton_rouge/sports...,-23.0,40.0,US,TheAdvocate,positive_anticipation_including_optimism,medium,"[[""season begins this week with expectations, ...","[[""high school football""]]","[[""season begins this week with expectations, ...","[[""excitement""]]",positive_surprise,no_i_did_not_feel_any_emotion
4998,f4ae5a03,100 years on: the picture that changed our vie...,https://www.theguardian.com/science/2019/may/1...,-6.0,48.0,UK,TheGuardian,positive_surprise,medium,"[[""the picture that changed our view of the un...",[[]],"[[""the picture that changed our view of the un...","[[""changed""]]",annoyance,annoyance


Removing unnecessary columns

In [65]:
df.drop(['id', 'url', 'country', 'source', 'cause', 'experiencer', 'target',
         'cue', 'bias_horizontal', 'bias_vertical', 'intensity', 'other_emotions',
         'reader_emotions'], axis = 1, inplace=True)
df

,headline,dominant_emotion
0,Cops in One Village Have Been Convicted of 70 ...,anger
1,DIY penis enlargements are a 'nationwide probl...,negative_surprise
2,Dam breaking: New Epstein accuser comes forward,anger
3,David Beckham gets six-month driving ban for u...,negative_surprise
4,Dead sea turtle found with spear through head ...,sadness
...,...,...
4995,‘NOW do abortion. We’ll wait.’ Ilhan Omar doub...,negative_surprise
4996,‘Triad’ Thugs Use Clubs to Punish Hong Kong’s ...,negative_surprise
4997,'For real': High school football season begins...,positive_anticipation_including_optimism
4998,100 years on: the picture that changed our vie...,positive_surprise


In [66]:
df.dropna(inplace=True)
df['dominant_emotion'].value_counts()

dominant_emotion
negative_surprise                            893
positive_surprise                            582
anger                                        460
annoyance                                    426
fear                                         419
sadness                                      386
disgust                                      382
negative_anticipation_including_pessimism    323
positive_anticipation_including_optimism     319
joy                                          264
guilt                                        175
trust                                        124
shame                                        119
pride                                         65
love_including_like                           63
Name: count, dtype: int64

Removing emotions that are not included in Paul Ekman's 6 basic emotions

In [67]:
#removing other dominant emotions other than ekman's basic 6 {except suprise}
emotions_to_keep = ['anger', 'fear', 'sadness', 'disgust', 'joy']
df_ekman = df.query('dominant_emotion in @emotions_to_keep')
print(df_ekman['dominant_emotion'].value_counts())
df_ekman

dominant_emotion
anger      460
fear       419
sadness    386
disgust    382
joy        264
Name: count, dtype: int64


,headline,dominant_emotion
0,Cops in One Village Have Been Convicted of 70 ...,anger
2,Dam breaking: New Epstein accuser comes forward,anger
4,Dead sea turtle found with spear through head ...,sadness
5,Democrats defend Congresswomen Trump says shou...,disgust
10,Donald Trump’s ‘bizarre’ questions on George W...,disgust
...,...,...
4968,HHS Moves to Protect Religious Health Providers,fear
4977,New Trump Rule Permits Anti-LGBTQ Health Care ...,disgust
4982,Robotic Hacks: CNNers Sound Awfully Similar Ba...,disgust
4988,Trump administration misses census printing de...,disgust


<h1>Data Cleaning</h1>

In [68]:
df_ekman['headline'] = df_ekman['headline'].str.lower()

In [69]:
def clean_text(headline):
    nopunc = [w for w in headline if w not in string.punctuation]
    nopunc = ''.join(nopunc)
    return  ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])

In [70]:
clean_text(df_ekman['headline'][0])

'cops one village convicted 70 crimes here’s say'

In [71]:
df_ekman['length'] = df_ekman['headline'].apply(len)

In [72]:
df_ekman['headline'][0], clean_text(df_ekman['headline'][0])

('cops in one village have been convicted of 70 crimes. here’s what they had to say about it.',
 'cops one village convicted 70 crimes here’s say')

In [73]:
df_ekman['headline'].head().apply(clean_text)

0       cops one village convicted 70 crimes here’s say
2        dam breaking new epstein accuser comes forward
4     dead sea turtle found spear head florida natio...
5     democrats defend congresswomen trump says “go ...
10    donald trump’s ‘bizarre’ questions george wash...
Name: headline, dtype: object

In [74]:
df_ekman['headline'] = df_ekman['headline'].astype(str)

In [75]:
def preprocess(headline):
    return ' '.join([word for word in word_tokenize(headline) if word not in stopwords.words('english') and not word.isdigit() and word not in string.punctuation])

In [76]:
preprocess(df_ekman['headline'][0])

'cops one village convicted crimes ’ say'

In [77]:
df_ekman['headline'] = df_ekman['headline'].apply(preprocess)

In [78]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(headline):
    return ' '.join([lemmatizer.lemmatize(word) for word in headline.split()])
df_ekman["headline"] = df_ekman["headline"].apply(lambda headline: lemmatize_words(headline))

#Data Splitting
For Training and Testing/Validation data

In [79]:
#Splitting Dataset
x = df_ekman['headline']
y = df_ekman['dominant_emotion']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=0)  # 30% of data reserved for testing
print(x_train.head())

2594    un human right chief condemns saudi arabia ’ e...
1464    letter family murder victim need sustained sup...
387     celebration nipsey hussle 's life take place l...
4105    nurse dy trying save another woman drowning st...
2171     panic war iran info operation preserve iran deal
Name: headline, dtype: object


##Feature Extraction

In [80]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
#feature extraction using tf-idf

#instantiation of CountVectorizer and TfidfTransformer
count_vect = CountVectorizer()
tf_transformer = TfidfTransformer()

#x_train uses fit_transform(); fit will only be done on training data
x_train_extract = count_vect.fit_transform(x_train)
x_train_extract = tf_transformer.fit_transform(x_train_extract)

#x_test; only transform () will be done on testing data
x_test_extract = count_vect.transform(x_test)
x_test_extract = tf_transformer.transform(x_test_extract)

#Gridsearch tuning

In [87]:
from sklearn.model_selection import GridSearchCV

In [88]:
# defining parameter range
param_grid = {'C': [0.1, 1, 2, 3, 4, 5],
              'kernel': ['linear']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1)

# fitting the model for grid search
grid.fit(x_train_extract, y_train)

# print best parameter after tuning
print(grid.best_params_)
grid_predictions = grid.predict(x_test_extract)

# print classification report
print(classification_report(y_test, grid_predictions))
print(accuracy_score(y_test, grid_predictions))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'C': 1, 'kernel': 'linear'}
              precision    recall  f1-score   support

       anger       0.40      0.56      0.47       143
     disgust       0.34      0.33      0.33       105
        fear       0.43      0.46      0.45       119
         joy       0.63      0.28      0.38        87
     sadness       0.50      0.44      0.47       120

    accuracy                           0.43       574
   macro avg       0.46      0.41      0.42       574
weighted avg       0.45      0.43      0.43       574

0.43031358885017423


#Model Training

In [89]:
#making linearsvc model
svc = SVC(kernel = 'linear')

#training linearsvc model on training data
svc.fit(x_train_extract, y_train)

SVC(kernel='linear')

#Testing Model Performance On Overall Dataset

In [90]:
#predicting testing data split
test_predictions = svc.predict(x_test_extract)

#getting accuracy
print("Accuracy: ", accuracy_score(y_test, test_predictions, normalize = True))

Accuracy:  0.43031358885017423


In [91]:
input = "Pagasa: Dagupan, Aparri to bear 47 °C heat index Friday"

input = count_vect.transform([input])
input = tf_transformer.transform(input)

test2 = svc.predict(input)
print(test2)


['fear']
